In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2
#%matplotlib qt

In [2]:
from skdim import global_id
from skdim import local_id
import skdim
import multiprocessing as mp
import scipy
import numpy as np
import pandas as pd
import rpy2
import rpy2.robjects as ro
import rpy2.robjects.numpy2ri
import rpy2.robjects.packages as rpackages

from sklearn.neighbors import NearestNeighbors
import sklearn.datasets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

rpy2.robjects.numpy2ri.activate()
utils = rpackages.importr('utils')
#utils.install_packages('intrinsicDimension')
#utils.install_packages('ider')
intdimr = rpackages.importr('intrinsicDimension')
ider   = rpackages.importr('ider')
#r_base = rpackages.importr('base')

import sys
sys.path.append('/home/utilisateur/id-concentration/')
from estimators import *

In [217]:
def compute_betas(max_dim):
    #COMPUTE_BETAS Computes betas between one and max_dim. This code is based on the paper 
    #   
    #   Díaz, M., Quiroz, A., & Velasco, M. (2018). 
    #   Local angles and dimension estimation from data on manifolds.
    # 
    #   betas = compute_betas( max_dim ) Computes the variance of the angle between two Gaussian
    #   vectors in RR^d, it does so for each d between 1 and max_dim.
    betas = np.zeros(max_dim-1)
    betas[:2] = np.array([np.pi**2/12, np.pi**2/4 - 2])

    if (max_dim >= 4):
        for ii in range(4,max_dim+1):
            betas[ii-2] = betas[ii-4] - 2/(ii-2)**2

    betas = np.concatenate((np.array([np.pi**2/4]), betas))

    return betas[:,None]

def compute_cuts(max_dim):
    #COMPUTE_CUTS Computes the cuts that define the thresholds to decide between
    #   dimensions. This code is based on the paper 
    #   
    #   Díaz, M., Quiroz, A., & Velasco, M. (2018). 
    #   Local angles and dimension estimation from data on manifolds.
    #
    #   etas = compute_cuts(max_dim) Computes the cuts to decide between any
    #   dimension between 1 and max_dim.
    betas = compute_betas(max_dim)
    etas = np.zeros((len(betas)-1,1))
    for ii in range(len(etas)):
        etas[ii] = (betas[ii] + betas[ii+1])/2
    
    return etas

def compute_statistic(dataset, p):
    # COMPUTE_STATISTIC Computes the estimate of the variance of the angles
    #   between points of the form X-p where X belongs to the data set. This code is based on the paper 
    #   
    #   Díaz, M., Quiroz, A., & Velasco, M. (2018). 
    #   Local angles and dimension estimation from data on manifolds.
    #
    #   [V, E] = compute_statistic( dataset, p ) Computes the empirical
    #   variance V and expected value E of the angle between points in the
    #   dataset centered at p.
    #
    k = dataset.shape[1]

    dataset = dataset - np.repeat(p,k,axis=1) #centers the points
    if len(dataset) > 1:
        dataset=dataset[:,np.any(dataset,axis=0)] #deletes zero columns

    norms = np.sqrt(np.sum(dataset**2,axis=1))
    dot_prods = ((dataset.T@dataset)/(norms.T@norms)) #computes the normalized the cosines of the angles for every pair
    angles = np.arccos(np.triu(dot_prods)) #gets the upper triangular part of the matrix of cosines and computes the angle
    V = np.sum((angles - np.pi/2)**2/len(angles)) #computes the statistics
    E = np.mean(angles - np.pi/2)

    return V, E 

def get_center(dataset):
    # GET_CENTER Finds the most central point in the dataset. This code is based on the paper 
    #   
    #   Díaz, M., Quiroz, A., & Velasco, M. (2018). 
    #   Local angles and dimension estimation from data on manifolds.
    #
    #   indx = get_center( dataset ) Returns the index of the most central point with respect
    #   to a centrality measure described in the paper. 
    d,n = dataset.shape
    centr = np.zeros(n)
    vcentral = 0.5 - np.abs(0.5-((2*np.arange(1,n+1)-1)/(2*n)))
    for ii in range(d):
        icentral = np.argsort(dataset[ii,:])
        centr[icentral] = centr[icentral] + vcentral

    indx = np.argmax(centr)
    return indx

def normpdf(x, mu, sigma):
    return np.exp(-.5*((x-mu)/sigma)**2)/(sigma*np.sqrt(2*np.pi))
def evaluate_approx_density(x, data):
    #EVALUATE_APPROX_DENSITY Evaluates normal kernel at each component of x.
    #
    #   densities = evaluate_approx_density( x, data ) Evaluates the normal 
    #   distribution estimated with "data" at the different components of the
    #   vector x. 

    M = len(data)
    h = (4/(3*M))**(1/5)
    densities = normpdf(x,data,h)
    density = np.mean(densities)/h

    return density

In [131]:
np.random.seed(1337)
dataset=np.random.random((5,30)).T

In [139]:
nbr_centers=2
n=data.shape[1]

In [140]:
ind_cen

array([0, 2, 4])

In [169]:
np.random.seed(0)
np.random.choice(10,5,replace=False)

array([2, 8, 4, 9, 1])

In [229]:
def ANOVA_global_estimator(dataset)
    np.random.seed(0)
    D_max=10

    r_samples=[np.array([3,5,1])-1,np.array([1,2,3])-1]

    ind_cen = np.zeros(nbr_centers,dtype=int)
    for ii in range(nbr_centers):
        r_sample = np.random.choice(n,int(np.ceil(n/nbr_centers)),replace=False)
        indx = get_center(dataset[:,r_sample])
        ind_cen[ii] = r_sample[indx]

    centers = dataset[:,ind_cen]


###ANOVA local

def ANOVA_local_estimator(dataset, centers):
    #ANOVA_LOCAL_ESTIMATOR Estimates the dimension of the manifold in which
    #  the dataset lies at the center points. This code is based on the paper 
    #  
    #  Díaz, M., Quiroz, A., & Velasco, M. (2018). 
    #  Local angles and dimension estimation from data on manifolds.
    #  
    #  [d, first_moments ] = ANOVA_local_estimator( dataset, centers ) Estimate the dimension with
    #  for each one of the center points (where the centers are taken as columns of the matrix centers ). 
    #  Addtionally it gets an empirical expected value ('first_moments') of the angles between the neighbors of each center.  
    #
    #  [d, dh] = ANOVA_global_estimator( __ , Name, Value ) Estimate the dimension with
    #  the user-specified parameters. One can vary the numerical parameters number of
    #  neighbors 'k' and maximum possible dimension 'maxdimension'. The boolean parameter 'basic' determines what estimator to use between the basic
    #  estimator or the kernel-based one. The boolean parameter 'verbose'
    #  determines if the script prints a summary of the results or not.

    dimensions = np.zeros(centers.shape[1])
    first_moments = np.zeros(centers.shape[1])
    Us = np.zeros(centers.shape[1])
    betas = compute_betas(D_max)
    etas = compute_cuts(D_max) #basic version

    # Estimates the dimension for each center
    nn=NearestNeighbors(n_neighbors=k)
    nn.fit(centers.T)
    indicesknn = nn.kneighbors(dataset.T,return_distance=False)

    for ii in range(len(dimensions)):
        U, first_moments[ii] = compute_statistic(dataset[:,indicesknn[ii,:]], centers[:,[ii]])
        Us[ii] = U

        dd = 1
        while dd <= len(etas) and U < etas[dd]:
            dd+=1
        dimensions[ii] = dd

        if verbose:
            print('Center %d, dimension estimate %d'%(ii, dimensions[ii]))

    return dimensions, first_moments,indicesknn, Us

In [5]:
data = np.zeros((30,10))
data[:,:5] = skdim.gendata.hyperBall(n_points = 30, n_dim = 5, radius = 1, random_state = 0)

In [141]:
res2=radovanovic_estimators_matlab(data,k=10)